In [ ]:
# === colab configuration ===
# Skip this cell if running demo file locally!

from google.colab import drive
import sys

# setting paths
repository_path = '/content/time-series-forecasting-with-transformers/'
datasets_path = repository_path + 'datasets/'
sys.path.insert(0, repository_path)

# cloning project repository and downloading dataset
drive.mount('/content/drive')
! test ! -d $repository_path && git clone https://github.com/francescobaraldi/time-series-forecasting-with-transformers
# ! test ! -d $dataset_path && cp -R $dataset_path_drive $dataset_path
%cd $repository_path

# setting branch and pulling updates
branch = 'main'
! git checkout $branch
! git pull origin $branch

executing_on_colab = True

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from dataset import SP500Dataset, YahooDataset, YahooDatasetInference
from model import Transformer, TransformerDecoder, TransformerDecoder_v2, WeatherLSTM
from eval import eval_mae
from plot import plot_scores, plot_predictions
from train import train_model
from test import test

try:
    executing_on_colab
except NameError:
    executing_on_colab = False

## Set dataset paths

In [2]:
if not executing_on_colab:
    sp500_dataset_path = "datasets/spx.csv"
    yahoo_dataset_path = "datasets/yahoo_stock.csv"
    predictions_path = "predictions/"
    training_results_path = "training_results/"
else:
    sp500_dataset_path = datasets_path + "spx.csv"
    yahoo_dataset_path = datasets_path + "yahoo_stock.csv"
    predictions_path = "/content/drive/My Drive/time-series-forecasting-with-transformers/predictions/"
    training_results_path = "/content/drive/My Drive/time-series-forecasting-with-transformers/training_results/"

## Transformer std.

In [ ]:
# TODO

## Transformer decoder

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 32
learning_rate = 0.01
num_epochs = 50
window_len = 30
forecast_len = 7
input_size = 5
output_size = 1
d_model = 20

train_dataset = YahooDataset(yahoo_dataset_path, window_len, forecast_len, train=True)
scaler = train_dataset.get_scaler()
test_dataset = YahooDataset(yahoo_dataset_path, window_len, forecast_len, train=False, scaler=scaler)
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

model = TransformerDecoder(seq_len=window_len, num_layer=1, input_size=input_size, output_size=output_size, d_model=d_model, num_heads=1, feedforward_dim=32).to(device)
loss_fn = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model, results = train_model(device, model, train_dl, test_dl, num_epochs, loss_fn, eval_mae, optimizer)

plot_scores(results['train_scores'], results['test_scores'], results['losses'], training_results_path + "decoder" + "/")
inference_dataset = YahooDatasetInference(yahoo_dataset_path, window_len, scaler, forecast_len)
inference_dl = DataLoader(inference_dataset, batch_size=1, shuffle=False, drop_last=True)
test(device, model, inference_dl, output_size, scaler, save_path=predictions_path + "decoder" + "/")

## Transformer decoder v2

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 32
learning_rate = 0.01
num_epochs = 5
window_len = 7
forecast_len = 7
input_size = 10
output_size = 1

train_dataset = YahooDataset(yahoo_dataset_path, window_len, forecast_len, train=True)
scaler = train_dataset.get_scaler()
test_dataset = YahooDataset(yahoo_dataset_path, window_len, forecast_len, train=False, scaler=scaler)
train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, drop_last=True)

model = TransformerDecoder_v2(seq_len=window_len, num_layer=1, input_size=input_size, output_size=output_size, num_heads=1, feedforward_dim=32).to(device)
loss_fn = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

model, results = train_model(device, model, train_dl, test_dl, num_epochs, loss_fn, eval_mae, optimizer)

plot_scores(results['train_scores'], results['test_scores'], results['losses'], training_results_path + "decoder_v2" + "/")
inference_dataset = YahooDatasetInference(yahoo_dataset_path, window_len, scaler, forecast_len)
inference_dl = DataLoader(inference_dataset, batch_size=1, shuffle=False, drop_last=True)
test(device, model, inference_dl, output_size, scaler, save_path=predictions_path + "decoder_v2" + "/")

## LSTM model

In [ ]:
# TODO